In [28]:
import tensorflow as tf
import os
import sys
import go_datafile_reader
import model
import numpy as np
import model_eval
reload(model)
reload(go_datafile_reader)
reload(model_eval)

<module 'model_eval' from 'model_eval.py'>

In [2]:
train_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/train"
test_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/test"
ckpt_path = "/home/justin/Programming/GoAI/MovePredictionCNN/data/working/board_eval_cnn_5layer.ckpt"

In [3]:
train_files = []
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            train_files.append(filepath)

test_files = []
for subdir, dirs, files in os.walk(test_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            test_files.append(filepath)

print "num train: %d, num test: %d" %(len(train_files), len(test_files))

num train: 14724, num test: 149


In [4]:
#note you may have to change the os limit for number of open files to use the RandomAccessFileReader
#you can do this with the command "sudo ulimit -n 20000"
#if sudo can't find the ulimit command try the following below
#sudo sh -c "ulimit -n 20000 && exec su $LOGNAME"

#reader = go_datafile_reader.RandomAccessFileReader(train_files)
test_reader = go_datafile_reader.GoDatafileReader(test_files)

test_reader.num_epochs = 0
test_features = []
test_targets = []
test_move_numbers = []
while(test_reader.num_epochs == 0):
    final_state, _, feature_cube = test_reader.read_sample()
    test_move_numbers.append(test_reader.move_index)
    test_features.append(feature_cube)
    test_targets.append(final_state)

In [5]:
x, ownership = model.place_holders()
y_conv = model.model(x)
loss = model.loss_function(ownership, y_conv)
train_op = model.train_step(loss)

prediction = tf.round(y_conv)
correct_prediction = tf.equal(ownership, prediction)
correct_count = tf.reduce_sum(tf.cast(correct_prediction, "float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
saver.restore(sess, ckpt_path)

In [6]:
import matplotlib.pyplot as plt
import numpy as np

In [19]:
accuracies = []
y_trues = []
y_vals = []
y_preds = []
feature_cubes = []
for i in xrange(len(test_features)):
    if i%500 == 0:
        print i
    x_, y_ = test_features[i], test_targets[i]
    y_val, y_pred, acc = sess.run([y_conv, prediction, accuracy], feed_dict={x:[x_], ownership:[y_]})
    accuracies.append(acc)
    y_trues.append(np.reshape(y_, [19,19]))
    y_preds.append(np.reshape(y_pred[0], [19,19]))
    y_vals.append(np.reshape(y_val[0], [19,19]))
    feature_cubes.append(x_)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500


In [41]:
idx = 160
y_pred = y_preds[idx]
y_true = y_trues[idx]
y_val = y_vals[idx]
feature_cube = feature_cubes[idx]

print test_move_numbers[idx], accuracies[idx]
model_eval.print_info(feature_cube, y_pred, y_val,y_true)

    

161 0.822715
**************10*0*   1100000000000110001   9650000000013791106   1000000000000110000
*100***000***010011   1100000000000010011   9900000000002090065   1100000000000010011
*110**011000*0110**   1110000110001011010   9990000680008199064   1110000110000011001
*100*0***10110*1000   1100000011011001000   9900102579099029002   1100000111011001000
**1**100111**01*111   1111010011110011111   9995491199994099999   1110110011110011111
1*1*01110*1*111*1**   1111011111111110101   9996099999999993948   1110011111111111111
*10*0**010*****0*0*   0100000011111100100   4901054099999640813   1100010011111011111
*00****011111***0**   0000000011111100000   3002234099999551001   0000001011111011011
*******0011000*****   0000001001100000000   0123445009900032100   0000011001100011000
**0******00*****00*   0001010000000000000   0005465200013543002   0000001100000000000
*011************01*   0011010000000101010   0099463223443556093   0011011001000010011
**01**00*******11*1   0001000010000111111

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame()
df["Move Num"] = test_move_numbers
df["Accuracy"] = accuracies
grouped = df.groupby("Move Num").aggregate(np.mean)


In [16]:
grouped.plot()
plt.title("Accuracy by move number, 5layer 64filters")
plt.xlabel("Move Number")
plt.ylabel("Accuracy")
plt.show()

In [17]:
plot = plt.hist(df[df["Move Num"] == 250]["Accuracy"].values, bins = 40)
plt.title("Move 250 accuracy histogram, 5layer 64filters")
plt.xlabel("Accuracy")
plt.ylabel("Frequency")
plt.show()

In [15]:
bad_guys = df[(df["Move Num"] > 200) & (df["Accuracy"] < .70)]
print len(bad_guys)

60


In [32]:
bad_guys[:10]

,Move Num,Accuracy
296,296,0.531856
551,255,0.578947
801,250,0.565097
1109,308,0.584488
1396,287,0.479224
1666,270,0.562327
1934,268,0.506925
2216,282,0.570637
2716,256,0.545706
3012,296,0.487535
